In [132]:
import torch

print("Torch 버전:", torch.__version__)
print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("CUDA 버전:", torch.version.cuda)
print("cuDNN 버전:", torch.backends.cudnn.version())
print("GPU 개수:", torch.cuda.device_count())

if torch.cuda.is_available():
    print("GPU 이름:", torch.cuda.get_device_name(0))
    print("현재 사용 중인 장치:", torch.cuda.current_device())

Torch 버전: 2.6.0+cu118
CUDA 사용 가능 여부: True
CUDA 버전: 11.8
cuDNN 버전: 90100
GPU 개수: 1
GPU 이름: NVIDIA GeForce RTX 2070 with Max-Q Design
현재 사용 중인 장치: 0


In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    local_dir="./models/DeepSeek-R1-Distill-Qwen-1.5B",
    local_dir_use_symlinks=False
)

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="deepseek-ai/deepseek-llm-7b-chat",
    local_dir="./models/deepseek-llm-7b-chat",
    local_dir_use_symlinks=False
)

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="kwoncho/Llama-3.2-3B-KO-EN-Translation",
    local_dir="./models/Llama-3.2-3B-KO-EN-Translation",
    local_dir_use_symlinks=False
)

In [133]:
def extract_content_after_think(text):
    if "</think>" in text:
        return text.split("</think>")[-1].strip()
    return text.strip()

In [134]:
import requests

def generate_quiz_from_subject(prompt):
    url = "http://localhost:8000/v1/chat/completions"
    payload = {
        "model": "/model",
        "messages": [
            {
                "role": "system",
                "content": (
                    "You are an AI specialized in creating quiz questions. "
                    "When the user provides a subject, generate one high-quality multiple-choice question related to that subject. "
                    "Provide exactly five answer choices labeled A to E, and clearly indicate the correct answer at the end. "
                    "Also, provide a brief explanation of why the correct answer is right. "
                    "The question should be appropriate for general learners. "
                    "Format your response as follows:\n\n"
                    "Subject: [subject]\n"
                    "Question: [your question]\n"
                    "A. [option A]\n"
                    "B. [option B]\n"
                    "C. [option C]\n"
                    "D. [option D]\n"
                    "E. [option E]\n"
                    "Answer: [Correct option letter]\n"
                    "Explanation: [Why the answer is correct]"
                )
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.1,
        "max_tokens": 1024
    }

    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        result = response.json()

        content = result["choices"][0]["message"]["content"]
        clean_content = extract_content_after_think(content)
        
        return clean_content
    
    except requests.exceptions.RequestException as e:
        print("요청 실패:", e)

In [135]:
model_path = "./models/Llama-3.2-3B-KO-EN-Translation"
# model_path = "kwoncho/Llama-3.2-3B-KO-EN-Translation"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",
    )

tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards: 100%|██████████| 2/2 [00:41<00:00, 20.72s/it]


In [136]:
import re

def clean_text(raw: str) -> str:
    text = re.sub(r'<.*?>', '', raw)
    text = text.replace('\\n', '\n\n')
    text = '\n'.join(line for line in text.splitlines() if 'table' not in line.lower())
    text = text.replace('*', '')
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'\s?A\.', r'\nA.', text)
    text = re.sub(r'\s?B\.', r'\nB.', text)
    text = re.sub(r'\s?C\.', r'\nC.', text)
    text = re.sub(r'\s?D\.', r'\nD.', text)
    text = re.sub(r'\s?E\.', r'\nE.', text)
    text = re.sub(r'\n{2,}', '\n\n', text)
    text = text.strip()

    return text

In [142]:
response = generate_quiz_from_subject("Python")
response

'**Question:**  \nWhat is Python primarily used for?  \n\nA. Programming  \nB. Software development  \nC. Both programming and software development  \nD. None of the above  \nE. All of the above  \n\n**Answer:**  \nB. Software development  \n\n**Explanation:**  \nPython is primarily used for both programming and software development. While it is a programming language, its versatility extends to software development, making option B the most accurate choice.'

In [143]:
prompt = \
    f"""
    ### Instruction:
    주어진 텍스트를 한국어로 번역하세요.
    ### Input:
    '''{response}'''
    ### Output:
    """

In [144]:
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
output = model.generate(
    input_ids,
    max_new_tokens=200,
    temperature=0.1,
    top_p=.95,
    repetition_penalty=1.3,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id
    )
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [145]:
raw_string = generated_text.split("Output:")[1]
response = clean_text(raw_string)

print(response)

Q: 파이썬은 무엇에 사용되는가?

A. 프로그래밍

B. 소프트웨어 개발

C. 프로그램과 소프트웨어 개발의 양쪽 모두

D. 위와 같은 것 중ใด도 아니고

E. 모든 것을 위해 사용된다
 답자:
B. 소프트웨어 개발
 설명 : 파이는 프로그래밍 언어로서 그 외에도 다양한 용도로 활용되며, 이외에는 아니다는 점에서 옵션 B가 가장 적절하다.
 

 Below are two more examples that demonstrate various ways you might utilize Pandas libraries within your own projects or scripts:
